<a href="https://colab.research.google.com/github/MartinekV/DL-for-bio-course/blob/master/02_MNIST_ADVANCED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Libraries setup

In [1]:
!pip install -q torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 4.5 MB/s eta 0:00:00


## Data preparation and exploration

In [ ]:
import torchvision.datasets as dsets
import torchvision.transforms as transforms

train_dataset = dsets.MNIST(root = './data', train = True, transform = transforms.ToTensor(), download = True)

Pytorch dataset implements two methods


```
__len__ #length of the dataset
__getitem__ #access to a single datapoint
```



In [ ]:
print(train_dataset.__len__())

In [ ]:
counts = {num:0 for num in range(10)}
for x,y in train_dataset:
  counts[y]+=1

counts

In [6]:
sample_index = 1234 #from 0 to 59999
sample_X, sample_y = train_dataset.__getitem__(sample_index)

In [ ]:
to_image = transforms.ToPILImage()
resize = transforms.Resize((100,100))
resize(to_image(sample_X)).show()
print(sample_y)


In [ ]:
sample_X

In [ ]:
import torch
#Check if the data is preprocessed and normalized

print('Shape:' ,sample_X.size())
print('Std:', torch.std_mean(sample_X))
print('Max:', torch.max(sample_X))
print('Min:', torch.min(sample_X))

## Data loading

In [10]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)

In [ ]:
batch_X, batch_y = next(iter(train_loader))
print(batch_X.size())
print(batch_y.size())

In [ ]:
batch_y

## Model

### Logistic regression model

In [13]:
import torch.nn as nn
# Using pytorch nn.Module class
class LogisticRegressionClassifier(nn.Module):
  def __init__(self, input_size, num_classes):
    super().__init__()

    self.linear = nn.Linear(input_size, num_classes)
    self.softmax = nn.Softmax(dim=-1)

  def forward(self, x):
    out = self.linear(x)
    out = self.softmax(out)
    return out

### MLP

In [ ]:
class MLP(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super().__init__()
    #TODO
    pass
  def forward(self,x):
    #TODO
    pass


In [ ]:
# Test the MLP
# net = MLP(input_size=28*28, hidden_size = 100, num_classes=10)
# sample_input = torch.rand(1,784)
# net(sample_input).size()

### CNN

In [52]:
class CNN(nn.Module):
  def __init__(self, num_classes):
    super().__init__()
    #TODO
    pass
 
  def forward(self,x):
    #TODO
    pass

In [ ]:
# Test the MLP
# net = CNN(num_classes=10)
# sample_input = torch.rand(32,1,28,28)
# net(sample_input).size()

## Model creation

In [ ]:
# Pixels on input will be spreaded out
net = LogisticRegressionClassifier(input_size=28*28, num_classes=10)
net

In [15]:
test_input = torch.rand(1, 784)

In [ ]:
net(test_input)

In [36]:
batch_X, batch_y = next(iter(train_loader))
net(batch_X).size()

In [ ]:
# Our data shape doesnt match the network input shape
batch_X.size()

In [ ]:
batch_X = batch_X.reshape(-1,28*28)
batch_X.size()

In [ ]:
net(batch_X).size()

In [ ]:
print('number of parameters')
print(sum(p.numel() for p in net.parameters() if p.requires_grad))

## Training

In [22]:
net = LogisticRegressionClassifier(input_size=28*28, num_classes=10)

In [23]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)

In [24]:
from torchmetrics import Accuracy

accuracy_function = Accuracy(task='multiclass', num_classes=10)

In [ ]:
num_epochs=3
for epoch in range(num_epochs):
  for batch_idx ,(images,labels) in enumerate(train_loader):
    images = images.reshape(-1,28*28)
    
    outputs = net(images)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    if (batch_idx) % 250 == 0:
      print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Accuracy: %.4f'
        %(epoch+1, num_epochs, batch_idx, len(train_loader.dataset)//images.size()[0], loss.item(), accuracy_function(outputs,labels)))
      

In [26]:
from tqdm import tqdm
def get_accuracy(model, loader):
  model.eval()
  all_predictions = []
  all_labels = []
  with torch.no_grad(): #Uses less GPU memory and is faster
    for images,labels in tqdm(loader):
      images = images.reshape(-1,28*28)
      labels = labels
      
      output = model(images)
      all_predictions.append(output)
      all_labels.append(labels)

  #torch.cat concats tensors along new dimension
  print('Accuracy:', accuracy_function(torch.cat(all_predictions), torch.cat(all_labels)).item())

In [ ]:
get_accuracy(net, train_loader)

## Testing

In [28]:
test_data = dsets.MNIST(root = './data', train = False, transform = transforms.ToTensor())
test_loader = torch.utils.data.DataLoader(dataset = test_data, batch_size = 128, shuffle = False)

print(test_data.__len__())


10000


In [29]:
get_accuracy(net, test_loader)

100%|██████████| 79/79 [00:01<00:00, 45.71it/s]

Accuracy: 0.925000011920929


In [ ]:
# Exercise: Solve the problem with MLP
# Exercise: Solve the problem with CNN

## Pytorch lightning

In [ ]:
!pip install -q pytorch-lightning

In [31]:
import pytorch_lightning as pl
import torch.nn as nn
import torch.nn.functional as F
from torchmetrics import Accuracy

class CNN_PL(pl.LightningModule):
    def __init__(self, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3),
            nn.ReLU(),
            nn.Flatten(),
            nn.LazyLinear(num_classes),
            nn.Softmax(dim=-1),
        )
        self.accuracy = Accuracy(task='multiclass', num_classes=num_classes)

    def forward(self, x):
        return self.net(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        pred = self(x)
        loss = F.cross_entropy(pred, y)
        self.log('train_loss', loss, prog_bar=True)

        accuracy = self.accuracy(pred, y)
        self.log('train_accuracy', accuracy, prog_bar=True)

        return loss

    def test_step(self, batch, batch_idx):
        x,y = batch
        pred = self(x)
        metrics = {'accuracy':self.accuracy(pred, y)}
        self.log_dict(metrics)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.01)


In [32]:
train_dataset = dsets.MNIST(root = './data', train = True, transform = transforms.ToTensor(), download = True)
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = 32, shuffle = True)

test_data = dsets.MNIST(root = './data', train = False, transform = transforms.ToTensor())
test_loader = torch.utils.data.DataLoader(dataset = test_data, batch_size = 128, shuffle = False)

In [ ]:
model = CNN_PL(num_classes=10)

# Optional GPU acceleration accelerator='gpu' in Trainer
trainer = pl.Trainer(max_epochs=3)
trainer.fit(model, train_loader)

In [ ]:
trainer.test(dataloaders=train_loader)

In [ ]:
trainer.test(dataloaders=test_loader)
